In [ ]:
import csv
import random


with open('./dataset_austin_sentences.csv') as csv_file:
    reader = csv.DictReader(csv_file)
    sentences = list(reader)

In [ ]:
sentences_2016 = [sentence for sentence in sentences if sentence['year'] == '2016']
print('There are %d sentences in the 2016 survey' % len(sentences_2016))

In [ ]:
from debater_python_api.api.debater_api import DebaterApi
from debater_python_api.api.clients.keypoints_client import KpAnalysisUtils, KpaIllegalInputException
from austin_utils import init_logger
import os

init_logger()
api_key = os.environ['DEBATER_API_KEY']
debater_api = DebaterApi(apikey=api_key)
keypoints_client = debater_api.get_keypoints_client()
domain = 'stance_demo'
# keypoints_client.delete_domain_cannot_be_undone(domain)
try:
    domain_params = {'do_stance_analysis': True}
    keypoints_client.create_domain(domain, domain_params)
    print('domain was created')
except KpaIllegalInputException as e:
    print('domain already exist')

In [ ]:
from austin_utils import print_top_and_bottom_k_sentences

def get_top_quality_sentences(sentences, top_k, topic):    
    arg_quality_client = debater_api.get_argument_quality_client()
    sentences_topic = [{'sentence': sentence['text'], 'topic': topic} for sentence in sentences]
    arg_quality_scores = arg_quality_client.run(sentences_topic)
    sentences_and_scores = zip(sentences, arg_quality_scores)
    sentences_and_scores_sorted = sorted(sentences_and_scores, key=lambda x: x[1], reverse=True)
    sentences_sorted = [sentence for sentence, _ in sentences_and_scores_sorted]
    print_top_and_bottom_k_sentences(sentences_sorted, 10)
    return sentences_sorted[:top_k]

sentences_2016_top_1000_aq = get_top_quality_sentences(sentences_2016, 1000, 'Austin is a great place to live')

In [ ]:
def run_kpa(sentences_ids, run_params):
    future = keypoints_client.start_kp_analysis_job(domain=domain, 
                                                    comments_ids=sentences_ids, 
                                                    run_params=run_params)

    kpa_result = future.get_result(high_verbosity=True, 
                                   polling_timout_secs=5)
    
    return kpa_result

In [ ]:
sentences_texts = [sentence['text'] for sentence in sentences_2016_top_1000_aq]
sentences_ids = [sentence['id'] for sentence in sentences_2016_top_1000_aq]
keypoints_client.upload_comments(domain=domain, 
                                 comments_ids=sentences_ids, 
                                 comments_texts=sentences_texts)

keypoints_client.wait_till_all_comments_are_processed(domain=domain)

In [ ]:
from austin_utils import print_results

pos_result = run_kpa(sentences_ids, {'n_top_kps': 20, 'stances_to_run': ['pos'], 'stances_threshold': 0.5})
print_results(pos_result, n_sentences_per_kp=2, title='Positive results')

In [ ]:
neg_result = run_kpa(sentences_ids, {'n_top_kps': 20, 'stances_to_run': ['neg', 'sug'], 'stances_threshold': 0.5})
print_results(neg_result, n_sentences_per_kp=2, title='Negative results')

In [ ]:
pos_result = KpAnalysisUtils.set_stance_to_result(pos_result, 'pos')
neg_result = KpAnalysisUtils.set_stance_to_result(neg_result, 'neg')

In [ ]:
merged_result = KpAnalysisUtils.merge_two_results(pos_result, neg_result)

In [ ]:
print_results(merged_result, n_sentences_per_kp=2, title='Merged result')